In [ ]:
import sys

import folium
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
mpl.rcParams['figure.figsize'] = (8,6)
mpl.rcParams['axes.grid'] = False

# Setup the module path that contains the framework code
from pathlib import Path
module_path = str(Path.cwd())
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the libraries
from modules.DataOperations import *
from modules.DataProcessing import *
from modules.MachineLearner import *

In [2]:
inc_data_path = 'data/ecoregion/inc_count.csv'
pop_data_path = 'data/ecoregion/pop.csv'
env_data_path = 'data/ecoregion/env_data.pkl'
ecoregion_data_path = 'data/ecoregion/'

In [3]:
def plot_mean_performance_improvements(improve_df):

    improve_df = improve_df*100
    
    x = np.arange(3)
    model_labels = ['Linear', 'LSTM', 'GRU']
    
    width = 0.1
    plt.figure(num=None, figsize=(14,10))
    
    
    df_indices = improve_df.index.values
    xOffset = np.linspace(-0.35,0.35,8)
    markerFmts = ['o', '<', 's', '*', 'p', 'x', 'D', '^']
    labelArray = ['Fixed, $M^f$ = 5', 'Fixed, $M^f$ = 10', 'Fixed, $M^f$ = 20', 'Fixed, $M^f$ = 40', 'Detected, $\Delta_{MIN}$ = 5', 'Detected, $\Delta_{MIN}$ = 10', 'Detected, $\Delta_{MIN}$ = 20', 'Detected, $\Delta_{MIN}$ = 30']
    
    for i in range(len(df_indices)):
        plt.plot(x+xOffset[i], improve_df.loc[df_indices[i]].values, markerFmts[i], markersize = 10, label= labelArray[i])
    
    plt.xticks(ticks=x, labels=model_labels)
    plt.ylim([-10,40])
    plt.xlabel('Prediction Models')
    plt.ylabel('Improvement in Accuracy (%)')
    plt.legend(loc='upper right', ncol = 2)
#     plt.savefig('exports/predictions/window_methods/mean_improvements_ecoregion.svg')

def plot_mean_optimal_errors(opt_error_df):
    
    x = np.arange(3)
    model_labels = ['Linear', 'LSTM', 'GRU']
    
    width = 0.1
    plt.figure(num=None, figsize=(14,10))
    
    
    df_indices = opt_error_df.index.values
    xOffset = np.linspace(-0.35,0.35,8)
    markerFmts = ['o', '<', 's', '*', 'p', 'x', 'D', '^']
    labelArray = ['Fixed, $M^f$ = 5', 'Fixed, $M^f$ = 10', 'Fixed, $M^f$ = 20', 'Fixed, $M^f$ = 40', 'Detected, $\Delta_{MIN}$ = 5', 'Detected, $\Delta_{MIN}$ = 10', 'Detected, $\Delta_{MIN}$ = 20', 'Detected, $\Delta_{MIN}$ = 30']
    
    for i in range(len(df_indices)):
        plt.plot(x+xOffset[i], opt_error_df.loc[df_indices[i]].values, markerFmts[i], markersize = 10, label= labelArray[i])
    
    plt.xticks(ticks=x, labels=model_labels)
    plt.ylim([0,0.8])
    plt.xlabel('Prediction Models')
    plt.ylabel('Optimal MAE')
    plt.legend(loc='upper right', ncol = 2)
#     plt.savefig('exports/predictions/window_methods/mean_opt_mae_ecoregion.svg')

def find_improvemnts_over_no_pic(win_error_dict):
    metric_index = 1
    win_keys = list(win_error_dict.keys())
    
    win_opt_df = pd.DataFrame()
    
    linear_opt_error = []
    lstm_opt_error = []
    gru_opt_error = []

    linear_base_error = []
    lstm_base_error = []
    gru_base_error = []
    
    for w_key in win_keys:
        
        test_errors = win_error_dict[w_key]
        feature_counts = list(test_errors.keys())
        
        linear_errors = []
        lstm_errors = []
        gru_errors = []
        
        error_df = pd.DataFrame()

        
        for nPIC in feature_counts:
            linear_errors.append(test_errors[nPIC]['Linear'][metric_index])
            lstm_errors.append(test_errors[nPIC]['LSTM'][metric_index])
            gru_errors.append(test_errors[nPIC]['GRU'][metric_index])
        
        error_df['nFeat'] = feature_counts
        error_df['linear'] = linear_errors
        error_df['lstm'] = lstm_errors
        error_df['gru'] = gru_errors
        error_df = error_df.set_index('nFeat')
        
        opt_error = error_df.min()
        no_pic_error = error_df.loc[0]
        

        linear_opt_error.append(opt_error.linear)
        lstm_opt_error.append(opt_error.lstm)
        gru_opt_error.append(opt_error.gru)

        linear_base_error.append(no_pic_error.linear)
        lstm_base_error.append(no_pic_error.lstm)
        gru_base_error.append(no_pic_error.gru)
        
    win_opt_df['win_key'] = win_keys
    win_opt_df = win_opt_df.set_index('win_key')
    win_opt_df['linear_base_error'] = linear_base_error
    win_opt_df['lstm_base_error'] = lstm_base_error
    win_opt_df['gru_base_error'] = gru_base_error
    win_opt_df['linear_opt_error'] = linear_opt_error
    win_opt_df['lstm_opt_error'] = lstm_opt_error
    win_opt_df['gru_opt_error'] = gru_opt_error

    win_opt_df["linear_improve"] = -(win_opt_df["linear_opt_error"]-win_opt_df["linear_base_error"])/win_opt_df["linear_base_error"]
    win_opt_df["lstm_improve"] = -(win_opt_df["lstm_opt_error"]-win_opt_df["lstm_base_error"])/win_opt_df["lstm_base_error"]
    win_opt_df["gru_improve"] = -(win_opt_df["gru_opt_error"]-win_opt_df["gru_base_error"])/win_opt_df["gru_base_error"]
    
    return win_opt_df

In [23]:
# # Select an Ecoregion
eco_name = 'Bahia Coastal Forests'

# # Find all the locations inside that ecoregion with data
# locs_in_region_with_data = get_locations_in_ecoregion(eco_name)
# pop_df = get_population_data(locs_in_region_with_data)

# # Get incidence data for those locations
# inc_count_df = get_data_for_locs(locs_in_region_with_data, False )
# inc_count_df = inc_count_df[inc_count_df['date'] < '2019-12-22']

# # Save all data read from databases
# inc_count_df.to_csv(inc_data_path, index=False)
# pop_df.to_csv(pop_data_path, index=True)

# Read saved data
inc_count_df = pd.read_csv(inc_data_path)
pop_df = pd.read_csv(pop_data_path, index_col=0)

inc_count_df['date'] = pd.to_datetime(inc_count_df['date'])
inc_count_df.iloc[:, 1:] = inc_count_df.iloc[:, 1:].apply(pd.to_numeric)


# Combine the incidence data for the ecoregion
ecoregion_df= get_combined_df(pop_df['pop'], inc_count_df)

# Drop the locations that have inadequate data ( < 80%)
locs_to_drop = get_locs_below_threshold(0.8, inc_count_df)
inc_count_df.drop(locs_to_drop, axis=1, inplace=True)
locs_to_drop = [int(loc) for loc in locs_to_drop]
pop_df.drop(locs_to_drop, axis=0, inplace=True)


outbreak_sizes = inc_count_df.sum().sort_values(ascending = False)
inc_ratio_df = normalize_case_counts(inc_count_df, pop_df)
outbreak_ratios = inc_ratio_df.sum().sort_values(ascending = False)
add_case_counts(pop_df, outbreak_sizes, outbreak_ratios)

# combine the frames
inc_ratio_df['ecoregion'] = ecoregion_df['inc_ratio']

In [5]:
# # Get a centroid for the ecoregion
# lon = pop_df['lon'].mean()
# lat = pop_df['lat'].mean()
# print(f'Location coordinates (lat, lon): {lat}, {lon}')

# # Get station data
# print('Obtaining weather station data ...')
# daily_obs_df, dist = get_station_data(lat, lon, search_range=250, urlStation='http://10.7.168.148:3254/stations', urlObs='http://10.7.168.148:3254/observations', fromDate=inc_ratio_df['date'].min().date(), toDate=inc_ratio_df['date'].max().date(), tol=0.1)

# # Convert to match the incidence data
# Weather_obs_df = convert_weather_to_weekly(inc_ratio_df['date'], daily_obs_df)

# # Get reanalysis data
# print('Obtaining reanalysis data ...')
# Reanalysis_df = get_rean_data(lat, lon, inc_ratio_df['date'])
# obs_rean_df = pd.merge(Weather_obs_df, Reanalysis_df, how='outer', on='date')

# # Store the data
# obs_rean_df.to_pickle(env_data_path)

In [6]:
# Read Stored env data
obs_rean_df = pd.read_pickle(env_data_path)

In [15]:
# Parameters: correlation window
max_shift = 15
comp_win_ext = 4
phase_ext = 1
min_corr = 0.1

sort_options = {}
sort_options['corr_weight'] = True
sort_options['dist_weight'] = True
sort_options['prev_weight'] = True
sort_options['prev_type'] = 'relative'


# ML Parameters
# Prediction window parameters
output_steps = 4
input_steps = 8

# train eval test split: 50 30 20 percents
split_frac = [.5, .3, .2]
iterations = 150
verbosity = 0

target_loc = 'ecoregion'

feature_counts = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 20, 25, 30]

In [ ]:
# det win param
risk_min = 0.05
min_win_lengths = [5, 10, 20, 30]

det_win_ranked_pic_dict = {}
det_win_error_dict = {}
det_win_forecast_win_dict = {}

det_win_linear_model_dict = {}
det_win_lstm_model_dict = {}
det_win_gru_model_dict = {}

det_win_optimal_feat_dict = {}


for min_win_len in min_win_lengths:
    print('For minimum window length of: ' + str(min_win_len))
    
    print('Computing predictabilities')
    pic_predictabilities = compute_var_window_predictability(ic_id = str(target_loc), inc_df = inc_ratio_df, lag_weeks = max_shift, min_win_len=min_win_len, risk_min=risk_min, side_len = comp_win_ext, min_corr = min_corr, extend_phase = phase_ext)
    ranked_pics = sort_predictors_for_ecoregion(pic_predictabilities, pop_df, sort_options)
    
    det_win_ranked_pic_dict[min_win_len] = ranked_pics
    
    print('Running ML fitting and evaluations')
    test_error_dict, forecast_window_dict, linear_model_dict, lstm_model_dict, gru_model_dict = run_ml_predictions(target_loc, obs_rean_df, inc_ratio_df, ranked_pics, feature_counts, split_frac, input_steps, output_steps, iterations, verbosity)
    
    
    det_win_error_dict[min_win_len] = test_error_dict
    det_win_forecast_win_dict[min_win_len] = forecast_window_dict

    det_win_linear_model_dict[min_win_len] = linear_model_dict
    det_win_lstm_model_dict[min_win_len] = lstm_model_dict
    det_win_gru_model_dict[min_win_len] = gru_model_dict
    
    
    min_errors = find_best_performers(feature_counts, test_error_dict, 1)
    min_errors['loc_id'] = target_loc

    optimal_feat_dict = {}
    optimal_feat_dict.update(min_errors)
    
    det_win_optimal_feat_dict[min_win_len] = optimal_feat_dict

In [ ]:
# save simulation results to storage
# np.save(ecoregion_data_path+'det_win_ranked_pic_dict.npy', det_win_ranked_pic_dict)
# np.save(ecoregion_data_path+'det_win_error_dict.npy', det_win_error_dict)
# np.save(ecoregion_data_path+'det_win_optimal_feat_dict.npy', det_win_optimal_feat_dict)

# load simulation results from storage
det_win_ranked_pic_dict = np.load(ecoregion_data_path+'det_win_ranked_pic_dict.npy', allow_pickle='TRUE').item()
det_win_error_dict = np.load(ecoregion_data_path+'det_win_error_dict.npy', allow_pickle='TRUE').item()
det_win_optimal_feat_dict = np.load(ecoregion_data_path+'det_win_optimal_feat_dict.npy', allow_pickle='TRUE').item()

In [ ]:
# fixed
data_splits = [5,10,20,40]

fixed_win_ranked_pic_dict = {}
fixed_win_error_dict = {}
fixed_win_forecast_win_dict = {}

fixed_win_linear_model_dict = {}
fixed_win_lstm_model_dict = {}
fixed_win_gru_model_dict = {}

fixed_win_optimal_feat_dict = {}


for no_of_windows in data_splits:
    print('The number of windows to used: ' + str(no_of_windows))
    
    print('Computing predictabilities')
    pic_predictabilities = compute_predictability(ic_id = 'ecoregion', inc_df = inc_ratio_df, lag_weeks = max_shift, no_of_windows = no_of_windows, side_len = comp_win_ext, min_corr = min_corr, extend_phase = phase_ext)
    ranked_pics = sort_predictors_for_ecoregion(pic_predictabilities, pop_df, sort_options)
    
    fixed_win_ranked_pic_dict[no_of_windows] = ranked_pics
    
    print('Running ML fitting and evaluations')
    test_error_dict, forecast_window_dict, linear_model_dict, lstm_model_dict, gru_model_dict = run_ml_predictions(target_loc, obs_rean_df, inc_ratio_df, ranked_pics, feature_counts, split_frac, input_steps, output_steps, iterations, verbosity)
    
    
    fixed_win_error_dict[no_of_windows] = test_error_dict
    fixed_win_forecast_win_dict[no_of_windows] = forecast_window_dict

    fixed_win_linear_model_dict[no_of_windows] = linear_model_dict
    fixed_win_lstm_model_dict[no_of_windows] = lstm_model_dict
    fixed_win_gru_model_dict[no_of_windows] = gru_model_dict
    
    
    min_errors = find_best_performers(feature_counts, test_error_dict, 1)
    min_errors['loc_id'] = target_loc

    optimal_feat_dict = {}
    optimal_feat_dict.update(min_errors)
    
    fixed_win_optimal_feat_dict[no_of_windows] = optimal_feat_dict

In [ ]:
# save simulation results to storage
# np.save(ecoregion_data_path+'fixed_win_ranked_pic_dict.npy', fixed_win_ranked_pic_dict)
# np.save(ecoregion_data_path+'fixed_win_error_dict.npy', fixed_win_error_dict)
# np.save(ecoregion_data_path+'fixed_win_optimal_feat_dict.npy', fixed_win_optimal_feat_dict)

# load simulation results from storage
fixed_win_ranked_pic_dict = np.load(ecoregion_data_path+'fixed_win_ranked_pic_dict.npy', allow_pickle='TRUE').item()
fixed_win_error_dict = np.load(ecoregion_data_path+'fixed_win_error_dict.npy', allow_pickle='TRUE').item()
fixed_win_optimal_feat_dict = np.load(ecoregion_data_path+'fixed_win_optimal_feat_dict.npy', allow_pickle='TRUE').item()

In [ ]:
linear_opt_error = []
lstm_opt_error = []
gru_opt_error = []

linear_improve = []
lstm_improve = []
gru_improve = []

win_scheme = []

win_opt_df = find_improvemnts_over_no_pic(fixed_win_error_dict)
win_scheme = ['mf ' + str(x) for x in list(win_opt_df.index)]

linear_opt_error += list(win_opt_df.linear_opt_error.values)
lstm_opt_error += list(win_opt_df.lstm_opt_error.values)
gru_opt_error += list(win_opt_df.gru_opt_error.values)

linear_improve += list(win_opt_df.linear_improve.values)
lstm_improve += list(win_opt_df.lstm_improve.values)
gru_improve += list(win_opt_df.gru_improve.values)

win_opt_df = find_improvemnts_over_no_pic(det_win_error_dict)
win_scheme = win_scheme + ['dm ' + str(x) for x in list(win_opt_df.index)]

linear_opt_error += list(win_opt_df.linear_opt_error.values)
lstm_opt_error += list(win_opt_df.lstm_opt_error.values)
gru_opt_error += list(win_opt_df.gru_opt_error.values)

linear_improve += list(win_opt_df.linear_improve.values)
lstm_improve += list(win_opt_df.lstm_improve.values)
gru_improve += list(win_opt_df.gru_improve.values)


opt_error_df = pd.DataFrame()
opt_error_df['schemes'] = win_scheme

opt_error_df['linear'] = linear_opt_error
opt_error_df['lstm'] = lstm_opt_error
opt_error_df['gru'] = gru_opt_error

opt_error_df['linear_improve'] = linear_improve
opt_error_df['lstm_improve'] = lstm_improve
opt_error_df['gru_improve'] = gru_improve

opt_error_df.set_index('schemes', inplace=True)

In [ ]:
plot_mean_performance_improvements(opt_error_df[['linear_improve', 'lstm_improve', 'gru_improve']])
plot_mean_optimal_errors(opt_error_df[['linear', 'lstm', 'gru']])

In [ ]:
nPIC = 4
min_win_len = 30
ranked_pics = det_win_ranked_pic_dict[min_win_len]
forecast_window = det_win_forecast_win_dict[min_win_len][nPIC]
ml_model = det_win_linear_model_dict[min_win_len][nPIC]


train_data, val_data, test_data, test_dates, train_mean, train_std, num_features = prepare_data(target_loc, obs_rean_df, inc_ratio_df, ranked_pics[0:nPIC], split_frac, False)

time_axis, label_array, prediction_array = forecast_window.combine_shifted_predictions(ml_model)

label_array = train_std['cases']*label_array + train_mean['cases']
prediction_array = train_std['cases']*prediction_array + train_mean['cases']

plt.figure(num=None, figsize=(20,6))
plt.scatter(time_axis, label_array, edgecolors='k', c='#2ca02c', s=64, label = 'Actual')
plt.scatter(time_axis, prediction_array, marker='X', edgecolors='k', c='#ff7f0e', s=64, label = 'Prediction')
plt.legend(loc='upper right')
plt.xlabel('Time')
plt.ylabel('Dengue cases per 100k')

In [ ]:
import folium.plugins as foplug
from sklearn.preprocessing import MinMaxScaler

nSelect = 40
picSelect = np.arange(0,nSelect)

# Select a bunch of points (based on pic ranks)
heatData = []
for idx in picSelect:
    loc_id = ranked_pics[idx]
    pic_data = pop_df.loc[int(loc_id)]
    nPIC = ranked_pics.shape[0]
    weight = (nPIC-idx)/nPIC
    
    heatData.append([pic_data.lat, pic_data.lon, weight])


heatTimeData = [heatData]*len(prediction_array)
heatTimeData = np.array(heatTimeData)

for idx, val in enumerate(prediction_array):
    heatTimeData[idx,:,2] = heatTimeData[idx,:,2]*val

scaler = MinMaxScaler(feature_range=(0.3,.8))
heatTimeData[:,:,2] = scaler.fit_transform(heatTimeData[:,:,2])

eco_coords = [pop_df['lat'].mean(), pop_df['lon'].mean()]
map_handle = folium.Map(location = eco_coords, zoom_start = 9, tiles='cartodbpositron')

# Ecoregion database not accessible at this moment, hence the boundary can't be drawn
# region = get_ecoregion_boundaries(eco_name)
# styleDict = {'fillColor': 'blue', 'color': 'red', 'weight': 2, 'opacity':0.3, 'fillOpacity':0 }
# folium.GeoJson(
#     region,
#     name='geojson',
#     style_function = lambda x: styleDict
# ).add_to(map_handle)

# hm = foplug.HeatMap(heatData)
# hm.add_to(map_handle)

taxis = np.array(pd.to_datetime(time_axis).strftime('%Y-%m-%d')).tolist()
hm = foplug.HeatMapWithTime(data = heatTimeData.tolist(), index = taxis, radius = 25)
hm.add_to(map_handle)

f = folium.Figure(width=800, height=600)
f.add_child(map_handle)